### Vamos Generalizar a otimização da carteira

Vamos ver como se sai o modelo com ativos norte americanos, e a ideia é nós aplicarmos posteriormente com ativos do mercado brasileiro.
Vou selecionar 4 ações, a ideia que eu tenho é que essas ações são potencialmente rentáveis, porém não sei quanto investir em cada uma delas e como mitigar os meus riscos.




In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go 
import yfinance as yf


tickers = ['XOM' , 'MCD', 'BRK-B' , 'GOOGL'] #definimos os tickers das ações 
## Ações 
prices = yf.download(tickers=tickers , start = '2017-01-01', end='2020-01-01')['Adj Close']
prices.dropna(inplace=True)

returns = (prices/prices.shift(1))
returns.dropna(inplace=True)
ativos = returns.columns.to_list()
returns


[*********************100%%**********************]  4 of 4 completed


Ticker,BRK-B,GOOGL,MCD,XOM
Date,,,,
2017-01-04,1.001526,0.999703,0.998830,0.988997
2017-01-05,0.995246,1.006499,1.001841,0.985093
2017-01-06,1.000674,1.014994,1.008856,0.999436
2017-01-09,0.991494,1.002387,0.997267,0.983503
2017-01-10,0.996605,0.998586,0.998505,0.987247
...,...,...,...,...
2019-12-24,0.999956,0.995409,1.002395,0.996159
2019-12-26,1.004346,1.013418,1.001983,1.001571
2019-12-27,0.998631,0.994253,1.005633,0.996578


### Distribuição dos retorno

Retorno líquido: 
$$Retorno = R = \dfrac{P_{t}}{P_{t-1}} - 1$$

Retorno "Bruto": 
$$R+1 = \dfrac{P_{t}}{P_{t-1}} $$


Podemos usar um outro formalismo que é o retorno logaritmo que simplifica os nossos cálculos

$$r_t = log(R+1) = log(P_t) - log(P_{t-1})$$

se o preço logaritmico for: $$p_t = log(P_t)$$

$$r_t   = p_t  - p_{t-1} $$

In [2]:
logReturns = np.log(returns)
logReturns

Ticker,BRK-B,GOOGL,MCD,XOM
Date,,,,
2017-01-04,0.001525,-0.000297,-0.001171,-0.011063
2017-01-05,-0.004765,0.006478,0.001840,-0.015019
2017-01-06,0.000673,0.014882,0.008817,-0.000565
2017-01-09,-0.008543,0.002384,-0.002737,-0.016635
2017-01-10,-0.003400,-0.001415,-0.001496,-0.012835
...,...,...,...,...
2019-12-24,-0.000044,-0.004601,0.002393,-0.003848
2019-12-26,0.004337,0.013329,0.001981,0.001570
2019-12-27,-0.001370,-0.005763,0.005617,-0.003428


## Sharpe Ratio

$$SR(w) = \dfrac{R(w) - R_f}{\sigma(w)}$$

Em que:
$$
w = \begin{bmatrix}
w_1 & w_2 & w_3 & w_4
\end{bmatrix}
$$

$$\sum_{i} ^{4} w_i = 1$$

é o vetor dos pesos normalizados.

- $SR(w)$: Sharpe ratio
- $R(f)$ : taxa livre de risco
- $R(w)$ : retorno do portfolio

$$R(w)  = w^{T} \mathbf{R} $$
$$\sigma(w) = \sqrt(w^{T}\Sigma w)$$

- $\Sigma$ é a matriz de covariância

Perceba que o máximo Sharpe ratio é tal que é proporcional à
$$SR(w) = \dfrac{\mathbf{r}}{\sqrt(w^{T}\Sigma w)}$$

In [3]:
### starting portfolio simulation
number_sim = 1000
weight = np.zeros((number_sim, 4))
mean_log_return  = logReturns.mean()
Sigma = logReturns.cov() # matriz de covariancia

expected_return = np.zeros(number_sim)
expected_risk = np.zeros(number_sim)
sharpe_ratio  = np.zeros(number_sim)





for k in range(number_sim):
    # gerar pesos aleatórios e normalizados
    w = np.array(np.random.random(4))
    w= w/np.sum(w)
    weight[k,:]= w 

    #retorno log esperado:
    expected_return[k] = np.sum( mean_log_return *w)

    # risco: 
    #expected_risk  = w.T @ Sigma @ w
    expected_risk[k]  = np.sqrt(np.dot(w.T, np.dot(Sigma, w)))

    sharpe_ratio[k]   = expected_return[k]/ expected_risk[k]




    



In [4]:
max_Index = sharpe_ratio.argmax()
weight[max_Index,:]

array([0.11318187, 0.29844921, 0.58195146, 0.00641746])